## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chuy,UY,-33.6971,-53.4616,49.41,77,14,2.75,few clouds
1,1,Touros,BR,-5.1989,-35.4608,75.16,83,97,9.22,overcast clouds
2,2,Carauari,BR,-4.8828,-66.8958,71.78,99,100,2.39,overcast clouds
3,3,Gat,IL,31.6100,34.7642,86.77,22,29,16.98,scattered clouds
4,4,Qaanaaq,GL,77.4840,-69.3632,7.86,71,52,8.34,broken clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 65


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']>=min_temp) & (city_data_df['Max Temp']<=max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chuy,UY,-33.6971,-53.4616,49.41,77,14,2.75,few clouds
5,5,Hermanus,ZA,-34.4187,19.2345,52.81,81,56,7.09,broken clouds
9,9,Taltal,CL,-25.4000,-70.4833,61.09,80,100,4.18,overcast clouds
11,11,Avila,ES,40.5833,-5.0000,48.65,76,62,2.53,broken clouds
12,12,Cerqueira Cesar,BR,-23.0356,-49.1661,62.58,87,72,14.85,broken clouds
14,14,Aras,ES,42.5617,-2.3560,48.61,87,0,3.44,clear sky
20,20,Cape Town,ZA,-33.9258,18.4232,56.12,91,20,5.75,few clouds
21,21,Port Elizabeth,ZA,-33.9180,25.5701,61.11,70,0,14.97,clear sky
22,22,Maun,BW,-19.9833,23.4167,51.76,100,0,1.14,clear sky
23,23,East London,ZA,-33.0153,27.9116,59.94,72,0,12.66,clear sky


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                210
City                   210
Country                204
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chuy,UY,-33.6971,-53.4616,49.41,77,14,2.75,few clouds
5,5,Hermanus,ZA,-34.4187,19.2345,52.81,81,56,7.09,broken clouds
9,9,Taltal,CL,-25.4000,-70.4833,61.09,80,100,4.18,overcast clouds
11,11,Avila,ES,40.5833,-5.0000,48.65,76,62,2.53,broken clouds
12,12,Cerqueira Cesar,BR,-23.0356,-49.1661,62.58,87,72,14.85,broken clouds


In [23]:
clean_travel_cities.count()

City_ID                204
City                   204
Country                204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
dtype: int64

In [24]:
clean_travel_cities.columns

Index(['City_ID', 'City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Current Description'],
      dtype='object')

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chuy,UY,49.41,few clouds,-33.6971,-53.4616,
5,Hermanus,ZA,52.81,broken clouds,-34.4187,19.2345,
9,Taltal,CL,61.09,overcast clouds,-25.4000,-70.4833,
11,Avila,ES,48.65,broken clouds,40.5833,-5.0000,
12,Cerqueira Cesar,BR,62.58,broken clouds,-23.0356,-49.1661,
14,Aras,ES,48.61,clear sky,42.5617,-2.3560,
20,Cape Town,ZA,56.12,few clouds,-33.9258,18.4232,
21,Port Elizabeth,ZA,61.11,clear sky,-33.9180,25.5701,
22,Maun,BW,51.76,clear sky,-19.9833,23.4167,
23,East London,ZA,59.94,clear sky,-33.0153,27.9116,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # combining coordinates under location for params dictionary
    params["location"] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chuy,UY,49.41,few clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
5,Hermanus,ZA,52.81,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
9,Taltal,CL,61.09,overcast clouds,-25.4000,-70.4833,Residencial Karime
11,Avila,ES,48.65,broken clouds,40.5833,-5.0000,El Carrascal
12,Cerqueira Cesar,BR,62.58,broken clouds,-23.0356,-49.1661,Pousada São Jorge
...,...,...,...,...,...,...,...
711,Radcha,UA,48.07,broken clouds,48.8502,24.6550,Pid Yavorom
715,Oriental,MX,61.23,overcast clouds,19.3667,-97.6167,Motel 111
716,Toora-Khem,RU,62.65,broken clouds,52.4675,96.1100,
720,Khiv,RU,54.95,overcast clouds,41.7543,47.9306,Four Seasons


In [28]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chuy,UY,49.41,few clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
5,Hermanus,ZA,52.81,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
9,Taltal,CL,61.09,overcast clouds,-25.4000,-70.4833,Residencial Karime
11,Avila,ES,48.65,broken clouds,40.5833,-5.0000,El Carrascal
12,Cerqueira Cesar,BR,62.58,broken clouds,-23.0356,-49.1661,Pousada São Jorge
14,Aras,ES,48.61,clear sky,42.5617,-2.3560,Jardines de Viana Spa
20,Cape Town,ZA,56.12,few clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
21,Port Elizabeth,ZA,61.11,clear sky,-33.9180,25.5701,39 On Nile Guest House
22,Maun,BW,51.76,clear sky,-19.9833,23.4167,Enviro Villa
23,East London,ZA,59.94,clear sky,-33.0153,27.9116,Tu Casa


In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10) 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chuy,UY,49.41,few clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
5,Hermanus,ZA,52.81,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
9,Taltal,CL,61.09,overcast clouds,-25.4000,-70.4833,Residencial Karime
11,Avila,ES,48.65,broken clouds,40.5833,-5.0000,El Carrascal
12,Cerqueira Cesar,BR,62.58,broken clouds,-23.0356,-49.1661,Pousada São Jorge
14,Aras,ES,48.61,clear sky,42.5617,-2.3560,Jardines de Viana Spa
20,Cape Town,ZA,56.12,few clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
21,Port Elizabeth,ZA,61.11,clear sky,-33.9180,25.5701,39 On Nile Guest House
22,Maun,BW,51.76,clear sky,-19.9833,23.4167,Enviro Villa
23,East London,ZA,59.94,clear sky,-33.0153,27.9116,Tu Casa


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description},{Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))